In [91]:
import pandas as pd
from pandas.tseries.frequencies import to_offset

import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import math

from sklearn.preprocessing import MinMaxScaler


import time
import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.metrics import mean_absolute_percentage_error

import warnings
warnings.filterwarnings('ignore')

In [92]:
data = pd.read_csv("./Dataset/STB.csv")
data.rename(columns={"trunc_time":"Date","open_price":"open","high_price":"high","low_price":"low","close_price":"Close"}, inplace= True)

In [93]:
from sklearn.preprocessing import MinMaxScaler
closedf = data[['Date','Close']]
print(closedf)
print("Shape of close dataframe:", closedf.shape)
close_stock = closedf.copy()
del closedf['Date']
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
print(closedf.shape)
time_step = 20

            Date  Close
0     2016-01-04  12600
1     2016-01-05  12300
2     2016-01-06  12800
3     2016-01-07  12600
4     2016-01-08  12600
...          ...    ...
1722  2022-11-21  16700
1723  2022-11-22  16900
1724  2022-11-23  17500
1725  2022-11-24  18200
1726  2022-11-25  18900

[1727 rows x 2 columns]
Shape of close dataframe: (1727, 2)
(1727, 1)


In [94]:
training_size=int(len(closedf)*0.8)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)
train_data

train_data:  (1381, 1)
test_data:  (346, 1)


array([[0.18563923],
       [0.17513135],
       [0.19264448],
       ...,
       [0.80210158],
       [0.76357268],
       [0.73905429]])

In [95]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)


In [96]:
# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 10
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)
print(test_data.shape)
print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)

(346, 1)
X_train:  (1370, 10)
y_train:  (1370,)
X_test:  (335, 10)
y_test (335,)


In [97]:
class Node:
    def __init__(self, split_feature=None, split_value=None, left=None, right=None, prediction=None, is_leaf=False):
        self.split_feature = split_feature
        self.split_value = split_value
        self.left = left
        self.right = right
        self.prediction = prediction
        self.is_leaf = is_leaf

In [98]:
import numpy as np

class DecisionTreeRegressor:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.tree = None
        
    def fit(self, X, y):
        self.tree = self._build_tree(X, y)
        
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.tree) for x in X])
    
    def _build_tree(self, X, y, depth=0):
        node = Node()
        
        if depth == self.max_depth or len(X) < self.min_samples_split or len(np.unique(y)) == 1:
            node.is_leaf = True
            node.prediction = np.mean(y)
            return node
        
        split_feature, split_value = self._find_best_split(X, y)
        
        if split_feature is None:
            node.is_leaf = True
            node.prediction = np.mean(y)
            return node
        
        left_indices = X[:, split_feature] <= split_value
        right_indices = X[:, split_feature] > split_value

        node.split_feature = split_feature
        node.split_value = split_value

        node.left = self._build_tree(X[left_indices], y[left_indices], depth + 1)
        node.right = self._build_tree(X[right_indices], y[right_indices], depth + 1)

        return node
    
    def _find_best_split(self, X, y):
        best_score = float('inf')
        best_feature = None
        best_value = None
        
        n_features = X.shape[1]
        if self.max_features is not None and self.max_features < n_features:
            feature_indices = np.random.choice(n_features, size=self.max_features, replace=False)
        else:
            feature_indices = np.arange(n_features)
        
        for feature in feature_indices:
            values = np.unique(X[:, feature])
            for value in values:
                score = self._calculate_split_score(X, y, feature, value)
                
                if score < best_score:
                    best_score = score
                    best_feature = feature
                    best_value = value
        
        return best_feature, best_value
    
    def _calculate_split_score(self, X, y, feature, value):
        left_indices = X[:, feature] <= value
        right_indices = X[:, feature] > value
        
        left_y = y[left_indices]
        right_y = y[right_indices]
        
        n_left = len(left_y)
        n_right = len(right_y)
        total_samples = n_left + n_right
        
        left_score = self._calculate_variance(left_y)
        right_score = self._calculate_variance(right_y)
        
        split_score = (n_left / total_samples) * left_score + (n_right / total_samples) * right_score
        return split_score
    
    def _calculate_variance(self, y):
        if len(y) == 0:
            return 0.0
        mean = np.mean(y)
        variance = np.mean((y - mean) ** 2)
        return variance
    
    def _traverse_tree(self, x, node):
        if node.is_leaf:
            return node.prediction
        else:
            if x[node.split_feature] <= node.split_value:
                return self._traverse_tree(x, node.left)
            else:
                return self._traverse_tree(x, node.right)


In [99]:
import numpy as np
from joblib import Parallel, delayed

class RandomForestRegressor:
    def __init__(self, n_estimators=10, max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features=None,
                 subsample_ratio=0.8, random_state=None, n_jobs=-1):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.subsample_ratio = subsample_ratio
        self.random_state = random_state
        self.n_jobs = n_jobs
        self.trees = []

    def fit(self, X, y):
        n_samples = len(X)
        n_subsample = int(self.subsample_ratio * n_samples)

        def fit_tree(i):
            subsample_indices = self._sgbt_subsample(n_samples, n_subsample)
            subsample_X = X[subsample_indices]
            subsample_y = y[subsample_indices]

            tree = DecisionTreeRegressor(max_depth=self.max_depth,
                                                                   min_samples_split=self.min_samples_split,
                                                                   min_samples_leaf=self.min_samples_leaf,
                                                                   max_features=self.max_features)
            tree.fit(subsample_X, subsample_y)
            return tree

        self.trees = Parallel(n_jobs=self.n_jobs)(
            delayed(fit_tree)(i) for i in range(self.n_estimators)
        )

    def predict(self, X):
        return np.mean([tree.predict(X) for tree in self.trees], axis=0)

    def _sgbt_subsample(self, n_samples, n_subsample):
        indices = np.arange(n_samples)
        np.random.RandomState(self.random_state).shuffle(indices)
        return indices[:n_subsample]

In [100]:
from sklearn.model_selection import KFold
import time
import psutil

# cross validation
# evaluate performance of model, can be used to be the target function in optuna
# ref: from sklearn.model_selection import cross_val_score
def cross_val_score(estimator, X, y, cv=5, scoring=None):
    scores = []
    
    # scoring: target function, if not provided it will be r2
    if scoring is None:
        scoring = r2_score
    
    cv_splitter = KFold(n_splits=cv, shuffle=True)
    
    for train_indices, test_indices in cv_splitter.split(X):
        X_train, X_test = X[train_indices], X[test_indices]
        y_train, y_test = y[train_indices], y[test_indices]
        
        estimator.fit(X_train, y_train)
        y_pred = estimator.predict(X_test)
        
        score = scoring(y_test, y_pred)
        scores.append(score)
    
    return np.mean(scores) # return average score of `cv` times run


def r2_score(y_true, y_pred):
    numerator = np.sum((y_true - y_pred) ** 2)
    denominator = np.sum((y_true - np.mean(y_true)) ** 2)
    r2 = 1 - (numerator / denominator)
    return r2

def measure_system_metrics():
    cpu_percent = psutil.cpu_percent()
    memory_usage = psutil.virtual_memory().percent
    disk_usage = psutil.disk_usage('/').percent
    network_io = psutil.net_io_counters()
    
    return cpu_percent, memory_usage, disk_usage, network_io

In [101]:
start_time = time.time()

rf = RandomForestRegressor(max_depth=26, min_samples_leaf=2, min_samples_split=10,n_estimators=160, max_features=3, subsample_ratio=0.8) # subsample_ratio: % split to subsampling, can be used to be the hyperparameter in optuna
scores = cross_val_score(rf, X_train, y_train, cv=5)
print("Cross-validated my model r2:", scores)
end_time = time.time()

# Measure system metrics after code execution
cpu_percent, memory_usage, disk_usage, network_io = measure_system_metrics()

execution_time = end_time - start_time

print(f"Execution time: {execution_time} seconds")
print(f"CPU usage: {cpu_percent}%")
print(f"Memory usage: {memory_usage}%")
print(f"Disk usage: {disk_usage}%")
print(f"Network I/O: {network_io}")

print("---------------------------------------------------------------------------------------------------------")


rf.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
# Lets Do the prediction 

RF_train_predict=rf.predict(X_train)
RF_test_predict=rf.predict(X_test)
# print("Train data prediction:", train_predict)
# # print("Test data prediction:", test_predict)
RF_train_predict = RF_train_predict.reshape(-1,1)
RF_test_predict = RF_test_predict.reshape(-1,1)

print("Train data prediction:", RF_train_predict.shape)
print("Test data prediction:", RF_test_predict.shape)

Train data prediction: (1370, 1)
Test data prediction: (335, 1)


In [ ]:
# Transform back to original form

RF_train_predict = scaler.inverse_transform(RF_train_predict)
RF_test_predict = scaler.inverse_transform(RF_test_predict)
RF_original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
RF_original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

In [ ]:
# Evaluation metrices RMSE and MAE
RF_RMSE_train = math.sqrt(mean_squared_error(RF_original_ytrain,RF_train_predict))
RF_MSE_train = mean_squared_error(RF_original_ytrain,RF_train_predict)
RF_MAE_train = mean_absolute_error(RF_original_ytrain,RF_train_predict)

RF_RMSE_test = math.sqrt(mean_squared_error(RF_original_ytest,RF_test_predict))
RF_MSE_test = mean_squared_error(RF_original_ytest,RF_test_predict)
RF_MAE_test = mean_absolute_error(RF_original_ytest,RF_test_predict)

print("Train data RMSE: ", RF_RMSE_train)
print("Train data MSE: ", RF_MSE_train)
print("Test data MAE: ", RF_MAE_train)
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", RF_RMSE_test)
print("Test data MSE: ", RF_MSE_test)
print("Test data MAE: ", RF_MAE_test)

Train data RMSE:  195.05908271961195
Train data MSE:  38048.045751416415
Test data MAE:  125.85801138338557
-------------------------------------------------------------------------------------
Test data RMSE:  2128.5088199235015
Test data MSE:  4530549.796492137
Test data MAE:  1713.5136338397267


In [ ]:
RF_EV_train = explained_variance_score(RF_original_ytrain, RF_train_predict)
RF_EV_test = explained_variance_score(RF_original_ytest, RF_test_predict)

print("Train data explained variance regression score:", RF_EV_train)
print("Test data explained variance regression score:", RF_EV_test)

Train data explained variance regression score: 0.9977513376304238
Test data explained variance regression score: 0.8515041343476395


In [ ]:
RF_r2_train = r2_score(RF_original_ytrain, RF_train_predict)
RF_r2_test = r2_score(RF_original_ytest, RF_test_predict)

print("Train data R2 score:", RF_r2_train)
print("Test data R2 score:", RF_r2_test)

Train data R2 score: 0.9977506126834059
Test data R2 score: 0.8299662736937686


In [ ]:
RF_MGD_train = mean_gamma_deviance(RF_original_ytrain, RF_train_predict)
RF_MGD_test = mean_gamma_deviance(RF_original_ytest, RF_test_predict)
RF_MPD_train = mean_poisson_deviance(RF_original_ytrain, RF_train_predict)
RF_MPD_test = mean_poisson_deviance(RF_original_ytest, RF_test_predict)
print("Train data MGD: ", RF_MGD_train)
print("Test data MGD: ", RF_MGD_test)
print("----------------------------------------------------------------------")
print("Train data MPD: ", RF_MPD_train)
print("Test data MPD: ",RF_MPD_test)

Train data MGD:  0.00017653632992864748
Test data MGD:  0.0060600828835020186
----------------------------------------------------------------------
Train data MPD:  2.4197957085721944
Test data MPD:  163.94140029791183


In [ ]:
# shift train predictions for plotting
from itertools import cycle
import plotly.express as px


print(RF_train_predict.shape)
print(RF_test_predict.shape)
print(len(closedf))

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(RF_train_predict)+look_back, :] = RF_train_predict
print("Train predicted data: ", trainPredictPlot)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(RF_train_predict)+(look_back*2)+1:len(closedf)-1, :] = RF_test_predict
print("Test predicted data: ", testPredictPlot)

close_stock['Predictions']=testPredictPlot
close_stock['Stock']='STB'
close_stock['Model']='DIY'
close_stock['Method']='None'
close_stock.to_csv('./output/STB_DIY_None.csv', index=False)

names = cycle(['Original close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['Date'],
                       'original_close': close_stock['Close'],
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

(1370, 1)
(335, 1)
1727
Train predicted data:  [[nan]
 [nan]
 [nan]
 ...
 [nan]
 [nan]
 [nan]]
Test predicted data:  [[           nan]
 [           nan]
 [           nan]
 ...
 [16723.85689484]
 [17175.33618552]
 [           nan]]
